In [1]:
from cabocha.analyzer import CaboChaAnalyzer
text_path = 'data/neko.txt'
cabocha_path = 'data/neko.txt.cabocha'

In [2]:
!cabocha -f1 < $text_path > $cabocha_path

### 40. 係り受け解析結果の読み込み（形態素）

In [3]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

In [4]:
def parseCabocha(block):
    lst = []
    for line in block.split('\n'):
        if line == '':
            return lst
        elif line[0] == '*':
            continue
        (surface, attr) = line.split('\t')
        attr = attr.split(',')
        lst.append(Morph(surface, attr[6], attr[0], attr[1]))

In [5]:
with open(cabocha_path, mode='rt', encoding='utf-8') as f:
    blockList = f.read().split('EOS\n')
blockList = list(filter(lambda x: x != '', blockList))
blockList = [parseCabocha(block) for block in blockList]
for m in blockList[2]:
    print(vars(m))

{'surface': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': 'まだ', 'base': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続'}
{'surface': '無い', 'base': '無い', 'pos': '形容詞', 'pos1': '自立'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}


### 41. 係り受け解析結果の読み込み（文節・係り受け）

In [6]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []

In [7]:
def parseCabocha(block):
    def checkCreateChunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            lst.append(c)
            tmp = []
        return tmp
    
    
    lst = []
    tmp = []
    dst = None
    for line in block.split('\n'):
        if line == '':
            tmp = checkCreateChunk(tmp)
        elif line[0] == '*':
            tmp = checkCreateChunk(tmp)
            dst = line.split(' ')[2].rstrip('D')
        else:
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            tmp.append(Morph(surface, attr[6], attr[0], attr[1]))
    for i, v in enumerate(lst):
        # 係り先なし
        if int(v.dst) == -1:
            continue
        lst[int(v.dst)].srcs.append(i)
    return lst

In [8]:
with open(cabocha_path, mode='rt', encoding='utf-8') as f:
    blockList = f.read().split('EOS\n')
blockList = list(filter(lambda x: x != '', blockList))
blockList = [parseCabocha(block) for block in blockList]
for m in blockList[7]:
    print([mo.surface for mo in m.morphs], m.dst, m.srcs)

['この'] 1 []
['書生', 'という', 'の', 'は'] 7 [0]
['時々'] 4 []
['我々', 'を'] 4 []
['捕え', 'て'] 5 [2, 3]
['煮', 'て'] 6 [4]
['食う', 'という'] 7 [5]
['話', 'で', 'ある', '。'] -1 [1, 6]


### 42. 係り元と係り先の文節の表示

In [9]:
def print_sentence(block):
    s = ''
    for m in block:
        for mo in m.morphs:
            s += mo.surface
    print(s)

In [10]:
for block in blockList[:10]:
    print_sentence(block)
    for m in block:
        if int(m.dst) > -1:
            s = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
            t = ''.join([mo.surface for mo in block[int(m.dst)].morphs if mo.pos != '記号'])
            print(f'{s}\t{t}')

一
　吾輩は猫である。
	猫である
吾輩は	猫である
名前はまだ無い。
名前は	無い
まだ	無い
　どこで生れたかとんと見当がつかぬ。
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
何でも	薄暗い
薄暗い	所で
じめじめした	所で
所で	泣いて
ニャーニャー	泣いて
泣いて	記憶している
いた事だけは	記憶している
吾輩はここで始めて人間というものを見た。
吾輩は	見た
ここで	始めて
始めて	人間という
人間という	ものを
ものを	見た
しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。
しかも	種族であったそうだ
あとで	聞くと
聞くと	種族であったそうだ
それは	種族であったそうだ
書生という	人間中で
人間中で	種族であったそうだ
一番	獰悪な
獰悪な	種族であったそうだ
この書生というのは時々我々を捕えて煮て食うという話である。
この	書生というのは
書生というのは	話である
時々	捕えて
我々を	捕えて
捕えて	煮て
煮て	食うという
食うという	話である
しかしその当時は何という考もなかったから別段恐しいとも思わなかった。
しかし	思わなかった
その	当時は
当時は	なかったから
何という	考も
考も	なかったから
なかったから	思わなかった
別段	恐し
恐し	思わなかった
いとも	思わなかった
ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。
ただ	載せられて
彼の	掌に
掌に	載せられて
載せられて	持ち上げられた
スーと	持ち上げられた
持ち上げられた	時
時	フワフワした
何だか	フワフワした
フワフワした	感じが
感じが	あったばかりである


### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [11]:
for block in blockList[:10]:
    for m in block:
        if int(m.dst) > -1:
            if any([mo.pos == '名詞' for mo in m.morphs]) and any([mo.pos == '動詞' for mo in block[int(m.dst)].morphs]):
                s = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
                t = ''.join([mo.surface for mo in block[int(m.dst)].morphs if mo.pos != '記号' ])
                print(f'{s}\t{t}')

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである


### 44. 係り受け木の可視化

In [12]:
!pip install pydot

In [29]:
import pydot

block = blockList[7]
pairs = []
for m in block:
    if int(m.dst) > -1:
        s = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
        t = ''.join([mo.surface for mo in block[int(m.dst)].morphs if mo.pos != '記号'])
        pairs.append([s, t])
g = pydot.graph_from_edges(pairs)
print(pairs)
g.write_png('data/graph.png', prog='dot')

[['この', '書生というのは'], ['書生というのは', '話である'], ['時々', '捕えて'], ['我々を', '捕えて'], ['捕えて', '煮て'], ['煮て', '食うという'], ['食うという', '話である']]
